# Домашнее задание

## Пункт 2

In [1]:
from pymystem3 import Mystem
import re

m = Mystem()

In [2]:
# чтение книги из файла и сохранение ее по параграфам
with open('book.txt', encoding='utf-8') as book:
    book = book.read()
    
    paragraphs = [paragraph for paragraph in book.split('\n') if len(paragraph)>0] 
    text = ''
    
    
# обработка параграфов
for paragraph in paragraphs:
#     оставляем только слова
    lemmas = [word for word in m.lemmatize(paragraph) if (len(re.findall(r'[а-яё]', word)) > 0)]
    
    for lemma in lemmas:
#         формируем текст с пробелами после слов и переносом строки после абзацев
        text += lemma + ' '
    
    text = text[:len(text) - 1] + '\n'

    
# записываем обработанные данные
with open('new_book.txt', 'w', encoding='utf-8') as new_file:
    new_file.write(text)

#### результат сохранен в файле new_book.txt

## Пункт 3

In [3]:
from nltk.tokenize import word_tokenize
import jsonlines
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

### Если при запуске следующего блока кода появляется ошибка с nltk word_tokenize, необходимо раскомментировать и запустить последующий блок кода

In [4]:
# из токенов оставляем только слова
# (словом считается та строка, в которой содержится хотя бы одна русская буква)
tokens = word_tokenize(book)
words = [word for word in tokens if (len(re.findall(r'[А-ЯЁа-яё]', word))>0)]

# создаем список, в котором буду лежать словари для каждого слова
dict_list = []
for word in words:
    dct = {}
    data = morph.parse(word)[0]
    dct['lemma'] = data.normal_form
    dct['word'] = word
    dct['pos'] = data.tag.POS
    dict_list.append(dct)

# записываем в файл с помощью  jsonlines
with open('tokens_dict_book.jsonl', 'w',encoding = 'utf-8') as output:
    with jsonlines.Writer(output) as writer:
        writer.write_all(dict_list)

In [5]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('punkt')

#### результат сохранен в файле tokens_dict_book.jsonl¶

## Пункт 4

### Часть 1

In [6]:
from collections import Counter

In [7]:
pos = [word['pos'] for word in dict_list]

# создаем списки для каждой части речи
print('Ответ:\n')

# рассчитываем и выводим долю слов каждой части речи в процентном соотношении
for key, val in Counter(pos).items():
    #обсчитываю с помощью counter
    print(key, ': ', round((val / len(pos)) * 100, 2), ' %', sep='')

Ответ:

NOUN: 30.74 %
ADJF: 11.8 %
VERB: 12.7 %
PREP: 10.78 %
CONJ: 9.8 %
ADVB: 6.32 %
GRND: 0.85 %
NPRO: 5.73 %
PRCL: 4.88 %
PRTF: 0.95 %
INFN: 2.54 %
ADJS: 0.63 %
NUMR: 0.78 %
COMP: 0.56 %
PRED: 0.28 %
PRTS: 0.58 %
INTJ: 0.02 %
None: 0.06 %


### Подпункт 2

#### глаголы

In [8]:
# составляем список глаголов
verbs = [word['lemma'] for word in dict_list if word['pos'] == 'VERB']

# сортируем список от большего к меньшему (порядок обозначетсязначениемв Counter, то есть кол-вом употреблений)
vlist = sorted(Counter(verbs).keys(), key=Counter(verbs).get, reverse=True)

# выводим первые 20
for i in range(20):
    print(vlist[i])

быть
стать
мочь
сказать
говорить
казаться
видеть
объявить
знать
оказаться
помнить
выйти
верить
хотеть
работать
пить
думать
следовать
сделать
стоять


#### наречия

In [9]:
# аналогично глаголам
adv = [word['lemma'] for word in dict_list if word['pos'] == 'ADVB']
advlist = sorted(Counter(adv).keys(), key=Counter(adv).get, reverse=True)

for i in range(20):
    print(advlist[i])

ещё
только
уже
теперь
несколько
очень
где
никогда
всегда
там
почти
тут
потом
хорошо
вдруг
сразу
наконец
иногда
вместе
ничего


## Пункт 5

### Биграммы

In [10]:
from nltk import bigrams

In [11]:
# считываем созданный ранее файл с леммами без пунктуации
with open('new_book.txt', encoding='utf-8') as f:
    text = f.read()

text = text.split()
# создем отсортированный от большего к меньшему список биграмм (ключ - значение в Counter, то есть количество употреблений)
text_bigrams = sorted(Counter(bigrams(text)).keys(), key = Counter(bigrams(text)).get, reverse = True)

# выводим первые 25
for i in range(25):
    print(text_bigrams[i])

('на', 'ферма')
('то', 'что')
('не', 'быть')
('зверский', 'ферма')
('что', 'они')
('они', 'не')
('и', 'не')
('не', 'мочь')
('что', 'он')
('мистер', 'джонс')
('никогда', 'не')
('весь', 'животное')
('в', 'этот')
('в', 'свой')
('это', 'быть')
('бывший', 'кабан')
('он', 'быть')
('свинья', 'и')
('и', 'даже')
('ферма', 'и')
('и', 'в')
('в', 'дом')
('они', 'быть')
('что', 'в')
('говорить', 'что')


### Триграммы

In [12]:
from nltk import trigrams

In [13]:
# аналогично биграммам
text_trigrams = sorted(Counter(trigrams(text)).keys(), key = Counter(trigrams(text)).get, reverse = True)

for i in range(25):
    print(text_trigrams[i])

('весь', 'животное', 'британия')
('битва', 'при', 'коровник')
('на', 'задний', 'ножка')
('в', 'то', 'что')
('о', 'то', 'что')
('четыре', 'хорошо', 'два')
('бывший', 'кабан', 'борька')
('снежок', 'и', 'наполеон')
('зверь', 'да', 'не')
('у', 'он', 'быть')
('боксер', 'и', 'кашка')
('хорошо', 'два', 'плохо')
('в', 'конец', 'конец')
('в', 'битва', 'при')
('свинья', 'и', 'пес')
('на', 'большой', 'гумно')
('один', 'из', 'они')
('с', 'тот', 'пора')
('как', 'раз', 'в')
('весь', 'зверь', 'равный')
('в', 'этот', 'минута')
('я', 'быть', 'работать')
('быть', 'работать', 'еще')
('работать', 'еще', 'упорно')
('зверский', 'ферма', 'и')


#### тут будет ответ про би и три граммы

## Пункт 6

В данной задаче важно понимать, что не все глаголы имеют все 3 формы и не все существительные могут употребляться во множественном и единственном числе. Так что программа подобные случаи типа pluralia / singularia tantum не обрабатывает.
(Подобные ошибки особо заметны на примере "миссис Джонс" -> "миссис Джонсы")

In [14]:
# указываем текст для обработки
paragraph = '''Владелец Барской Фермы мистер Джонс позапирал  на ночь курятники, но  о цыплячьих лазах спьяну забыл. Пошатываясь и рисуя на земле петли лучом света от фонарика, он пересек двор, скинул сапоги у  заднего крыльца, нацедил себе еще одну кружку пива из бочонка в буфетной при кухне и завалился на кровать, в которой уже похрапывала миссис Джонс.
Лишь только  свет  в спальне погас, вся усадьба пришла в  движение. 
Еще днем по ферме пронесся слух, будто прошлой ночью старый Майор, премированный хряк  средней белой  породы, видел поразительный сон и желает поведать о нем другим животным.'''

# разбиваем текст на токены для анализа
words = word_tokenize(paragraph)

# обрабатывает токены-слова
for word in words:
    new_word = word
    data = morph.parse(word)[0]

#     для глаголов прошедшее меняется на настоящее или будущее, а непрошедшее меняется на прошедшее
    if data.tag.POS == 'VERB':
        if data.tag.tense == 'past':
            if data.inflect({'pres'}):
                new_word = data.inflect({'pres'}).word
            else:
                new_word = data.inflect({'futr'}).word
        else:
            new_word = data.inflect({'past'}).word
            
#     для существительных единственное чсило меняется на множественное и наоборот
    if data.tag.POS == 'NOUN':
        if data.tag.number == 'sing' and data.inflect({'plur'}):
            new_word = data.inflect({'plur'}).word
        elif data.inflect({'sing'}):
            new_word = data.inflect({'sing'}).word
            
    #заменяем в тексте старые слова на новые
    paragraph = paragraph.replace(word, new_word)

# выводим текст
print(paragraph)

владельцы Барской ферм мистеры джонсы позапираю  на ночи курятник, но  о цыплячьих лазе спьяну забуду. Пошатываясь и рисуя на землях петель лучами света от фонариков, он пересеку дворы, скину сапог у  заднего крылец, нацежу себе еще одну кружки пива из бочонков в буфетной при кухнях и завалюсь на кровати, в которой уже похрапываю миссис джонсы.
Лишь только  свет  в спальнях погасну, вся усадьбы приду в  движения. 
Еще днями по фермах пронесусь слухи, будто прошлой ночию старый майоры, премированный хряки  средней белой  порода, вижу поразительный сны и желал поведать о нем другим животному.
